In [15]:
import pandas as pd
import numpy as np
from pymatgen.core import Composition
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from IPython.core.pylabtools import figsize
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
small_dataset = pd.read_csv('/data/single_perovskite_oxide.csv')
small_dataset = small_dataset.drop(columns = 'Unnamed: 0').reset_index(drop=True)
small_dataset = small_dataset.drop(columns = 'composition').reset_index(drop=True)
small_dataset['composition'] = small_dataset['formula'].map(Composition)
small_dataset

,formula,target,A,B,composition
0,SmCrO3,1.736200,Sm,Cr,"(Sm, Cr, O)"
1,ScCoO3,0.786000,Sc,Co,"(Sc, Co, O)"
2,GdCrO3,0.825000,Gd,Cr,"(Gd, Cr, O)"
3,YCrO3,1.767900,Y,Cr,"(Y, Cr, O)"
4,LaFeO3,0.848400,La,Fe,"(La, Fe, O)"
5,SmCoO3,0.803900,Sm,Co,"(Sm, Co, O)"
6,YFeO3,0.637300,Y,Fe,"(Y, Fe, O)"
7,LaCrO3,1.903900,La,Cr,"(La, Cr, O)"
8,PrCrO3,1.847000,Pr,Cr,"(Pr, Cr, O)"
9,NdCrO3,1.904000,Nd,Cr,"(Nd, Cr, O)"


In [8]:
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from pymatgen.core import Composition
feature_calculators = MultipleFeaturizer([cf.ElementProperty.from_preset("magpie"),
                                          cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])
feature_labels = feature_calculators.feature_labels()
small_magpie_features = feature_calculators.featurize_dataframe(small_dataset,col_id='composition')

MultipleFeaturizer:   0%|          | 0/52 [00:00<?, ?it/s]

In [9]:
basedata_magpie_features = small_magpie_features.copy(deep = True)
basedata_magpie_features = basedata_magpie_features.drop(columns = ['formula', 'target', 'A', 'B','composition'])
basedata_magpie_features

,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,MagpieData maximum MendeleevNumber,MagpieData range MendeleevNumber,MagpieData mean MendeleevNumber,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,8.0,62.0,54.0,22.0,16.80,8.0,23.0,87.0,64.0,66.6,...,86.2,89.04,12.0,1.8,2.4,1.0,1.2,True,0.724240,0.154892
1,8.0,27.0,19.0,14.4,7.68,8.0,11.0,87.0,76.0,66.0,...,84.8,87.36,12.0,2.0,2.4,1.6,0.0,True,0.660947,0.136622
2,8.0,64.0,56.0,22.4,17.28,8.0,27.0,87.0,60.0,67.4,...,91.8,95.76,12.0,1.8,2.4,1.2,1.4,True,0.714753,0.153484
3,8.0,39.0,31.0,17.4,11.28,8.0,12.0,87.0,75.0,64.4,...,91.8,95.76,12.0,1.8,2.4,1.2,0.0,True,0.708321,0.152541
4,8.0,57.0,49.0,21.4,16.08,8.0,13.0,87.0,74.0,65.8,...,91.8,95.76,12.0,2.0,2.4,1.4,0.0,True,0.745613,0.151694
5,8.0,62.0,54.0,22.6,17.52,8.0,23.0,87.0,64.0,68.4,...,79.2,80.64,12.0,2.0,2.4,1.4,1.2,True,0.724240,0.146339
6,8.0,39.0,31.0,17.8,11.76,8.0,12.0,87.0,75.0,65.6,...,91.8,95.76,12.0,2.0,2.4,1.4,0.0,True,0.708321,0.145782
7,8.0,57.0,49.0,21.0,15.60,8.0,13.0,87.0,74.0,64.6,...,91.8,95.76,12.0,1.8,2.4,1.2,0.0,True,0.745613,0.158143
8,8.0,59.0,51.0,21.4,16.08,8.0,17.0,87.0,70.0,65.4,...,91.8,95.76,12.0,1.8,2.4,1.0,0.6,True,0.736585,0.156756
9,8.0,60.0,52.0,21.6,16.32,8.0,19.0,87.0,68.0,65.8,...,91.8,95.76,12.0,1.8,2.4,1.0,0.8,True,0.733532,0.156291


In [10]:
from scipy.stats import spearmanr
def remove_collinear_features(x, y,threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model
        to generalize and improves the interpretability of the model.
        
    Inputs: 
        threshold: any features with correlations greater than this value are removed
    
    Output: 
        dataframe that contains only the non-highly-collinear features
    '''  
    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
#                 print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                x1 = x[col.values[0]]
                x2 = x[row.values[0]]
                corr1,p1 = spearmanr(x1, y)
                corr2,p2 = spearmanr(x2, y)
                if np.abs(corr1) < np.abs(corr2):
                    drop_cols.append(col.values[0])
                else:
                    drop_cols.append(row.values[0])
    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    return x

In [11]:
X_data = basedata_magpie_features
ymo_eg = small_dataset['target']
Xmo = remove_collinear_features(X_data,ymo_eg,0.8)
magpie_features_labels = Xmo.columns

In [12]:
Xmo = basedata_magpie_features[magpie_features_labels]
Xmo

,MagpieData mode Number,MagpieData maximum MendeleevNumber,MagpieData mode MendeleevNumber,MagpieData mode AtomicWeight,MagpieData minimum MeltingT,MagpieData maximum MeltingT,MagpieData range MeltingT,MagpieData mode MeltingT,MagpieData maximum Column,MagpieData mode Column,...,MagpieData mean GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg f valence electrons,compound possible,avg ionic char
0,8.0,87.0,87.0,15.9994,54.8,2180.00,2125.20,54.8,16.0,16.0,...,0.000000,0.0,12.0,86.2,89.04,12.0,1.8,1.2,True,0.154892
1,8.0,87.0,87.0,15.9994,54.8,1814.00,1759.20,54.8,16.0,16.0,...,0.309696,0.0,12.0,84.8,87.36,12.0,2.0,0.0,True,0.136622
2,8.0,87.0,87.0,15.9994,54.8,2180.00,2125.20,54.8,16.0,16.0,...,0.000000,0.0,12.0,91.8,95.76,12.0,1.8,1.4,True,0.153484
3,8.0,87.0,87.0,15.9994,54.8,2180.00,2125.20,54.8,16.0,16.0,...,0.000000,0.0,12.0,91.8,95.76,12.0,1.8,0.0,True,0.152541
4,8.0,87.0,87.0,15.9994,54.8,1811.00,1756.20,54.8,16.0,16.0,...,0.422133,0.0,12.0,91.8,95.76,12.0,2.0,0.0,True,0.151694
5,8.0,87.0,87.0,15.9994,54.8,1768.00,1713.20,54.8,16.0,16.0,...,0.309694,0.0,12.0,79.2,80.64,12.0,2.0,1.2,True,0.146339
6,8.0,87.0,87.0,15.9994,54.8,1811.00,1756.20,54.8,16.0,16.0,...,0.422133,0.0,12.0,91.8,95.76,12.0,2.0,0.0,True,0.145782
7,8.0,87.0,87.0,15.9994,54.8,2180.00,2125.20,54.8,16.0,16.0,...,0.000000,0.0,12.0,91.8,95.76,12.0,1.8,0.0,True,0.158143
8,8.0,87.0,87.0,15.9994,54.8,2180.00,2125.20,54.8,16.0,16.0,...,0.000000,0.0,12.0,91.8,95.76,12.0,1.8,0.6,True,0.156756
9,8.0,87.0,87.0,15.9994,54.8,2180.00,2125.20,54.8,16.0,16.0,...,0.000000,0.0,12.0,91.8,95.76,12.0,1.8,0.8,True,0.156291


In [17]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
magpie_etr = ExtraTreesRegressor(n_jobs=-1,random_state=42)
cv_indices = kf.split(Xmo)
error1 = []
error2 = []
r2_prm = []
for i, (train_index, test_index) in enumerate(cv_indices):
    X_train, X_test = Xmo.iloc[train_index], Xmo.iloc[test_index]
    y_train, y_test = ymo_eg[train_index], ymo_eg[test_index]
    name = f"fold_{i}"
    magpie_etr.fit(X_train, y_train)
    y_pred = magpie_etr.predict(X_test)
    mae_model = mean_absolute_error(y_test, y_pred)
    rmse_model = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_model = r2_score(y_test,y_pred)
    error1.append(mae_model)
    error2.append(rmse_model)
    r2_prm.append(r2_model)
    print(name)
    print(f"Performance on the %d set: MAE = %0.4f,RMSE = %0.4f, R2 = %0.4f" % (i,mae_model, rmse_model, r2_model))
mean_error1 = sum(error1) / len(error1)
mean_error2 = sum(error2) / len(error2)
mean_r2 = sum(r2_prm) / len(r2_prm)
print('Performance on the test set using 5-fold: MAE = %0.4f' % mean_error1)
print('Performance on the test set using 5-fold: RMSE = %0.4f' % mean_error2)
print('Performance on the test set using 5-fold: R2 = %0.4f' % mean_r2)

fold_0
Performance on the 0 set: MAE = 0.4745,RMSE = 0.6776, R2 = 0.3592
fold_1
Performance on the 1 set: MAE = 0.3946,RMSE = 0.5607, R2 = 0.6158
fold_2
Performance on the 2 set: MAE = 0.5132,RMSE = 0.6584, R2 = 0.3481
fold_3
Performance on the 3 set: MAE = 0.5464,RMSE = 0.7082, R2 = 0.5731
fold_4
Performance on the 4 set: MAE = 0.3740,RMSE = 0.5033, R2 = 0.6899
Performance on the test set using 5-fold: MAE = 0.4605
Performance on the test set using 5-fold: RMSE = 0.6216
Performance on the test set using 5-fold: R2 = 0.5172
